In [ ]:
import sys
sys.path.append("../src")
import importlib
from IPython.display import Audio
import sounddevice as sd
import wave
import time
from scipy.signal import butter, filtfilt
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
import logic.utils.io_access as io
import logic.audio_systems.device_helpers as dh
import logic.audio_systems.audio_transformers as at
import logic.audio_systems.speech_audio_stream_observable as saso

In [ ]:
# force reload imports
importlib.reload(dh)
importlib.reload(at)
importlib.reload(saso)


In [ ]:
device_index = dh.find_seed_device_index()
device_index

In [ ]:
devices = sd.query_devices()
target_description = "seeed-2mic-voicecard"
for i, device in enumerate(devices):
    if device['name'].startswith(target_description):
        print(device)

In [ ]:
input_channels_count = 2
output_channels_count = 1
sample_rate = 44100
record_seconds = 5
testfile_path = io.get_path('data', 'test_audio_pipeline.wav')

In [ ]:
# Record audio internally for notebook
class AudioDataObserver:
    def __init__(self, duration):
        self.filename = testfile_path
        self.rate = sample_rate
        self.channels = output_channels_count
        self.duration = duration
        self.binary_audio_data = bytearray()
        self.frames = int(sample_rate * duration)
        self.frame_count = 0

    def on_received(self, audio_data):
        self.binary_audio_data.extend(audio_data)
    
    # this is binary int16 data
    def get_binary_audio_data(self):
        return self.binary_audio_data
    
    def clear_audio_data():
        self.binary_audio_data = bytearray()
        self.frame_count = 0

In [ ]:
# record 5 seconds of audio
audio_stream_observable = saso.SpeechAudioStreamObservable()
audio_stream_observer = AudioDataObserver(record_seconds)
audio_stream_observable.add_observer(audio_stream_observer)

# Start the audio stream
try:
    print("Recording audio for 3 seconds...")
    time.sleep(record_seconds)
finally:
    print("Done recording.")
    audio_stream_observable.stop()
    
# Get recording binary
binary_recording_data = audio_stream_observer.get_binary_audio_data()

In [ ]:
# Convert to numpy array waves
np_input = at.bytes_to_int16(binary_recording_data)

# Play the audio
display(Audio(np_input, rate=sample_rate))


In [ ]:
# Visualize recoding
# Create the plots
fig, axs = plt.subplots(2, 1, figsize=(10, 6))

# Plot left channel
axs[0].plot(np_input)
axs[0].set_title('Left Channel')
axs[0].set_xlabel('Sample number')
axs[0].set_ylabel('Amplitude')

# Plot right channel
axs[1].plot(np_input)
axs[1].set_title('Right Channel')
axs[1].set_xlabel('Sample number')
axs[1].set_ylabel('Amplitude')

plt.tight_layout()
plt.show()

In [ ]:
print("Min after processing:", np.min(np_input))
print("Max after processing:", np.max(np_input))
print("Input data type:", np_input.dtype)

